In [15]:
%matplotlib inline
import IPython.core.display         
# setup output image format (Chrome works best)
IPython.core.display.set_matplotlib_formats("svg")
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
from scipy import stats
random.seed(100)
import pandas as pd
import csv

In [16]:
features_name=[
    # player 1 = big blind bets, always action player
    'round',
    'p1_card_1_number', # 0~12
    'p1_card_1_suit', # d:0, c:1, h:2, s:3
    'p1_card_2_number',
    'p1_card_2_suit',
    'p2_card_1_number',
    'p2_card_1_suit',
    'p2_card_2_number',
    'p2_card_2_suit',
    't1_p1_call', # yes:1, no:0
    't1_p1_fold',
    't1_p1_raise',
    't1_p1_raise_bets',
    't1_p2_call',
    't1_p2_fold',
    't1_p2_raise',
    't1_p2_raise_bets',
    
    't2_p1_call',
    't2_p1_fold',
    't2_p1_raise',
    't2_p1_raise_bets',
    't2_p2_call',
    't2_p2_fold',
    't2_p2_raise',
    't2_p2_raise_bets',
    
    't3_p1_call',
    't3_p1_fold',
    't3_p1_raise',
    't3_p1_raise_bets',
    't3_p2_call',
    't3_p2_fold',
    't3_p2_raise',
    't3_p2_raise_bets',
    
    't4_p1_call',
    't4_p1_fold',
    't4_p1_raise',
    't4_p1_raise_bets',
    't4_p2_call',
    't4_p2_fold',
    't4_p2_raise',
    't4_p2_raise_bets',
    
    
]

In [42]:
suit_values = {
    'd':0,
    'c':1,
    'h':2,
    's':3,
}

num_values = {
    '2':0,
    '3':1,
    '4':2,
    '5':3,
    '6':4,
    '7':5,
    '8':6,
    '9':7,
    'T':8,
    'J':9,
    'Q':10,
    'K':11,
    'A':12,
}

rows = []
f = open("C:\\Users\\User\\Desktop\\logs_2pn_2017\\ASHE.ElDonatoro.1.0.log", "r")
for line in f:
    # igonre not STATE
    if not line.startswith('STATE'):
        continue
        
    state, rounds, all_actions, all_cards, bets, names = line.split(':')
    print(state, rounds, all_actions, all_cards, bets, names )
    
    # read actions
    actions = all_actions.split('/')
    l = len(actions)
    for i in l:
        
    
    
    # read cards
    cards = all_cards.split('/')
    p1_hards, p2_hards = cards[0].split('|')
    p1_card_1_number = num_values[p1_hards[0]]
    p1_card_1_suit = suit_values[p1_hards[1]]
    p1_card_2_number  = num_values[p1_hards[2]]
    p1_card_2_suit = suit_values[p1_hards[3]]
    p2_card_1_number = num_values[p2_hards[0]]
    p2_card_1_suit = suit_values[p2_hards[1]]
    p2_card_2_number = num_values[p2_hards[2]]
    p2_card_2_suit = suit_values[p2_hards[3]]
    
    
    
    
    
    
    
    
    rows.append([
        int(rounds), 
        p1_card_1_number, 
        p1_card_1_suit, 
        p1_card_2_number, 
        p1_card_2_suit, 
        p2_card_1_number, 
        p2_card_1_suit, 
        p2_card_2_number, 
        p2_card_2_suit, 
    ])
    
    
    

STATE 0 r200c/r400c/r1200c/r20000c 6d2d|3s4d/KhJhTh/9s/6h 20000|-20000 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 1 r200c/cr400c/cr1200c/cr20000c Jd5c|Js8h/Qd6hQh/3c/Qc -20000|20000 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2 r300c/cc/r600c/r1800c Jc7c|9c5d/8hQdKc/5h/4d -1800|1800 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 3 r300f Kc3d|Ah3h -100|100 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 4 r300c/r900c/r1800c/r3600c KdJd|7sAs/JcTs7c/Ah/4d -3600|3600 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 5 r200c/cc/cr400c/cr1200c 6h6s|As9d/Ks7h4c/6d/8s 1200|-1200 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 6 r200c/cc/cc/r400c 2hAc|Kc8s/Qc6sJh/7s/7d 400|-400 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 7 r200c/cc/cr400c/cr800c Qd4d|8hQc/JsTh2c/8d/6s -800|800 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 8 r200c/cc/cc/r600c 5cJh|3c9c/9h3h5s/8c/Jd 600|-600 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 9 r300r900f Ad2d|5cJs 300|-300 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 10 r300c/cc/cc/r600c Tc4c|AcKh/AhQc5h/4d/3d -600|600

STATE 262 r200f 4d8h|Jc7d -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 263 r200r1000f 9d9c|5dAh 200|-200 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 264 r400f 6d3d|5sKh -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 265 r300c/cc/cc/cc 6cAh|ThKc/2s8hJd/3s/7s 300|-300 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 266 r300c/cc/cc/cc 3sKs|4sJc/9h7c5c/3c/5d 300|-300 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 267 r200c/cc/cc/cc TdJh|5cTh/4sQs7d/2h/6d 200|-200 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 268 r300f 9s6s|7hKd -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 269 r300c/cc/cc/cr20000c AsTh|Ac6s/8d9s9h/5s/7d -20000|20000 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 270 r200f 8s3c|5hQc -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 271 r200c/cc/cc/cc 5c8s|Qh2h/TsAcQs/9s/Js 200|-200 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 272 r300c/cc/cc/cc 8dJh|JcQc/2c8cQd/3d/5d -300|300 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 273 f 9c4h|5c2s 50|-50 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 274 r400f 6hA


STATE 506 r200c/cc/cc/r400c 5dQc|AsTd/QhKh6c/8d/3d 400|-400 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 507 r300f 4dKh|KsAh -100|100 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 508 f Qs9s|6c2h 50|-50 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 509 r200c/cc/cc/cc Kc7h|Jd4d/3s8hTs/Qc/5c 200|-200 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 510 r400f Ts8h|Kh3h -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 511 r200c/cc/cc/cc KcJs|5hKh/4d8s9c/6h/Qs 200|-200 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 512 r300f 5s6s|Ah7s -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 513 r300c/cc/cr900c/cr2700c 6cKh|4hAc/3c7h9c/Ah/Qc -2700|2700 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 514 r200c/cc/cc/cc Qh4c|7dTc/As8cJd/9c/7h -200|200 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 515 r300r1500f KdQd|8hTh 300|-300 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 516 r300f 2h5h|5s2s -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 517 r200r800f 4cTc|4s6h 200|-200 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 518 r200c/cc/cc/cc Ts2s|7s

STATE 755 f 8c8h|3s7d 50|-50 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 756 r300c/cc/r900c/r2700c 3cKs|As7d/2d8c7h/Kc/Js 2700|-2700 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 757 r200c/cc/cc/cr600c Ah2c|6c7c/Jd6s4d/Jh/5c -600|600 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 758 r200r400c/cc/cc/cc 8s8h|2hTh/AcAhKd/As/Jh 400|-400 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 759 r200f 7c3d|TdAd -100|100 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 760 r300f 5d9d|Td9s -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 761 r300c/cc/cr900c/cr20000c 5h9h|Qh4d/Qs9d7h/4c/Ah -20000|20000 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 762 r300f 2d8s|9h4h -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 763 r300f 7h4c|5sJd -100|100 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 764 r300f 5sJd|Js6c -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 765 f Qc5d|2h5c 50|-50 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 766 r300f 8s2d|5cTc -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 767 r200f 8h4d|5c6s -100|100 ElDonatoro_2pn_201

STATE 1005 r200f Tc2h|7h8h -100|100 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 1006 r200c/cc/cc/cc ThKh|As4d/Qc3h7c/Qs/Ac -200|200 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 1007 r200r1000f 5cJh|Ts8c 200|-200 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 1008 r200c/r400c/cc/cc 5s7d|9hAs/7hJd5h/Jc/Td 400|-400 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 1009 r200r800f 7h8h|2hTc 200|-200 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 1010 cc/cc/cc/cc 9c5d|Th2s/Kd4cAd/Qc/8d -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 1011 r300f 3hKc|KhQc -100|100 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 1012 r200f 5h4s|3cJs -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 1013 r300c/cc/cc/cr900c Kc8s|6c4d/8c6sTh/Kh/4h 900|-900 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 1014 r300c/cc/cc/cc 7cAh|Ts8h/3h3s9h/Js/2c 300|-300 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 1015 r200r1000f JcJs|5c6h 200|-200 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 1016 f 3h8h|6d2s 50|-50 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 1017 r200f 3h9c|4hKs

...

STATE 2005 f QsAc|5d3h 50|-50 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2006 r300f 2d8c|TsAs -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2007 f 3dKc|2c6h 50|-50 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2008 cr300f 8h8c|9d4s 100|-100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2009 r300r1200r2400f 6h5d|QsQh -1200|1200 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2010 r200c/cc/cc/cc 9d5c|8dQd/9s7hJd/Tc/3h -200|200 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2011 f AsAc|8d2h 50|-50 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2012 cc/cc/cc/cc 2h4d|JcKd/Ac8h9c/2c/Kc -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2013 r300c/cc/cc/cc 5sAh|Td6h/5c8d9h/4h/2d 300|-300 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2014 f 2c5s|2h6d 50|-50 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2015 r200c/cc/cc/cr20000c Td4d|4s6s/Qd8h8s/3s/As -20000|20000 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2016 cc/cc/cc/cc 6c8h|Tc5d/Kd4cQd/6d/5s 100|-100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2017 r300c/cc/cc/cr900c 8s7c|Kh2s/Kd3s7

STATE 2255 r300f 6sJh|Ah7d -100|100 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2256 r200c/cc/cc/cc KdQd|4sTd/7h4c3h/2h/3d -200|200 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2257 r200c/cc/cc/cc 7s6d|Js6s/3c9dAc/Qs/2s -200|200 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2258 r200r600f JdJh|6cTh 200|-200 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2259 r200c/cc/cc/cc Jh6d|7h4d/8s5d3c/Ah/8d 200|-200 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2260 r400f 3c7d|5sKh -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2261 r300f 5c2h|Qd6s -100|100 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2262 r200c/cc/cc/cc Jd8s|JcAs/8h2c9c/9h/Qs 200|-200 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2263 r300c/cc/cc/cc Jc9d|Tc7d/Kc4d2d/5d/6s 300|-300 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2264 cr200c/cc/cc/cc 5cJc|Qc7s/7h4sQh/Ks/9c -200|200 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2265 r300c/cr900c/cr2700c/cr20000c As8c|3d4d/3s5s3c/Jh/Qs -20000|20000 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2266 r400c/r1200c/r3600c/r20000c Ad

STATE 2505 r300c/cc/cc/cc 8c7h|4d4s/3dAcTs/3c/Kh -300|300 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2506 r300c/cc/cc/cc 6cKc|7s5s/Tc9dAh/3c/As 300|-300 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2507 r300c/cc/cc/cr900c TcQd|JhKh/4h4d6h/8d/Ks -900|900 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2508 f Qc4h|6d2h 50|-50 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2509 r200c/cc/cc/cc 4h5s|2cAh/5hJh4s/7h/Ks 200|-200 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2510 r300f 4c3d|As5c -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2511 r300c/cc/cc/cc ThTs|8hQs/5c3s5h/Ad/6c 300|-300 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2512 cr200c/cc/cc/cc Qc7h|Kh9d/4cThKc/4d/Js -200|200 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2513 r300f 4d8c|TsKd -100|100 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2514 f 4dJh|2c8d 50|-50 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2515 r200c/cc/cr600c/cr1800c 4sJh|5sAh/Kh9d3d/Ac/7c -1800|1800 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2516 r400f Td5c|JhKh -100|100 ASHE_2pn_2017|ElDonatoro_

STATE 2671 r300c/cc/cc/cc 7d8s|KcQh/Ad3dTs/5h/9h -300|300 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2672 r300c/cc/r900c/r20000c JcTd|3d6c/8cQs6h/9d/Js 20000|-20000 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2673 r200r600c/cc/cc/cc 6d6c|4d4s/5sAh3s/Td/9s 600|-600 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2674 r300f Th3h|9sKc -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2675 r200c/cc/cc/cc 6c4d|7dKh/8c9s4h/Qd/4s 200|-200 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2676 r200c/cc/cc/cc 7cAc|Kh5s/7dQc9s/Td/3d 200|-200 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2677 r300r1200c/cc/cc/cc 8s5s|9dAh/9cTdQc/Jh/Ts 1200|-1200 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2678 r400f 3sTc|Ad7d -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2679 r300f 6c4h|Kd6h -100|100 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2680 r200f 4h6c|Js9c -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2681 r300c/cc/cc/cc 6dKc|4h6s/AsJsTh/8c/Ts 300|-300 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2682 cc/cc/cc/cc 3h4s|2sJs/7cKd9s/3c

STATE 2921 r200r600f KsJc|4hJd 200|-200 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2922 cr300r1500f Jd3d|6dQh -300|300 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2923 r300f 4c9h|Qh9c -100|100 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2924 r300c/cc/cc/cc 7cJh|8sAh/9s6d3d/7h/Tc -300|300 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2925 r200c/cc/cc/cc 4sTc|Jc6s/Qc8dJd/8c/3c -200|200 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2926 r200c/cc/cc/r20000c TcKs|6sJs/4s9h8h/Qs/Jh 20000|-20000 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2927 f ThKd|4h3s 50|-50 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2928 f Ac5h|9d3s 50|-50 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2929 r300f 7c2s|5hQc -100|100 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2930 r200f 2h4h|3s3c -100|100 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2931 r300c/cc/cc/cr20000c 6dQd|5cAh/6sTcTd/Kc/As -20000|20000 ElDonatoro_2pn_2017|ASHE_2pn_2017

STATE 2932 cr200c/cc/cc/r600c JhTs|6h8c/4cKdKs/2h/Tc 600|-600 ASHE_2pn_2017|ElDonatoro_2pn_2017

STATE 2933 r300c/

In [12]:
print(rows[0])

['0', 'r200c/r400c/r1200c/r20000c', '6d2d|3s4d/KhJhTh/9s/6h', '20000|-20000', 'ASHE_2pn_2017|ElDonatoro_2pn_2017\n']


In [38]:
print(rows[0])

[0, 4, 0, 0, 0, 1, 3, 2, 0]


In [39]:
# write CSV file
f = open('converted_1.csv', 'w')
writer = csv.writer(f)
writer.writerows(rows)
f.close()